# Topic Modeling

In [1]:
%load_ext autoreload
%autoreload 2
from cord import ResearchPapers
import pandas as pd

In [2]:
research_papers = ResearchPapers.load()

Loading metadata from ..\data\CORD-19-research-challenge
Cleaning metadata

Indexing research papers
Creating the BM25 index from the abstracts of the papers
Use index="text" if you want to index the texts of the paper instead
Finished Indexing in 51.0 seconds


In [ ]:
research_papers.save()

In [ ]:
import gensim
from gensim.models import LdaModel
from gensim.models.ldamulticore import LdaMulticore

dictionary = gensim.corpora.Dictionary(research_papers.index_tokens)
corpus =  [dictionary.doc2bow(text) for text in research_papers.index_tokens]

In [ ]:
%time lda_model: LdaModel = LdaMulticore(corpus, num_topics=6, passes=4)

In [ ]:
def get_topic(tokens):
    print(tokens)
    bow = dictionary.doc2bow(tokens)
    tp =lda_model[bow]
    return tp

get_topic(research_papers.index_tokens[0])

In [ ]:
lda_model.get_topics()

## Coherence Scores

In [ ]:
from gensim.models.coherencemodel import CoherenceModel

def get_lda_model(papers, num_topics=4, passes=4):
    dictionary = gensim.corpora.Dictionary(papers.index_tokens)
    corpus =  [dictionary.doc2bow(text) for text in papers.index_tokens]
    lda_model: LdaModel = LdaMulticore(corpus, num_topics=num_topics, passes=4)
    return lda_model

def calculate_coherence_score(documents, dictionary, model):
    coherence_model = CoherenceModel(model=model, texts=documents, dictionary=dictionary,coherence='c_v')
    return coherence_model.get_coherence()

def get_coherence_scores(papers, start, stop):
    dictionary = gensim.corpora.Dictionary(papers.index_tokens)
    corpus =  [dictionary.doc2bow(text) for text in papers.index_tokens]
    scores = []
    for num_topics in range(start, stop):
        print('Calculating coherence for', num_topics, 'topics')
        l_model = LdaMulticore(corpus, num_topics=num_topics, id2word=dictionary, passes=6)
        coherence = calculate_coherence_score(papers.index_tokens, dictionary, l_model)
        scores.append({'topics':num_topics, 'coherence': coherence})
    return pd.DataFrame(scores)
        

min_topics, max_topics = 4, 15
coherence_scores = get_coherence_scores(research_papers.since_sarscov2(), min_topics,  max_topics)
alt.Chart(coherence_scores).mark_line().encode(
    x = 'topics',
    y = 'coherence'
)

In [ ]:
scores = pd.DataFrame({'score': coherence_scores, 
                       'topics': list(range(min_topics, max_topics))})
scores

In [ ]:
def get_lda_model(papers, num_topics=6, passes=4):
    dictionary = gensim.corpora.Dictionary(papers.index_tokens)
    corpus =  [dictionary.doc2bow(text) for text in papers.index_tokens]
    lda_model = LdaMulticore(corpus, num_topics=num_topics, id2word=dictionary, passes=passes)
    return lda_model

In [ ]:
a = [(0, 0.2), (1, 0.1), (2, 0.4), (3, 0.05), (4, 0.25)]
a.sort(key=lambda tup: tup[1], reverse=True)
